In [2]:
import sklearn as sk
import pandas as pd
import warnings
warnings.simplefilter("ignore")

In [3]:
# First, lets construct the different models that we will use.
from sklearn import tree
from sklearn import neighbors
from sklearn import naive_bayes
from sklearn import svm
from sklearn import neural_network
from sklearn import ensemble

# Read data-engineered CSV.
df = pd.read_csv('H-2B_Engineered_Data.csv')
labels = df.loc[:,'CASE_STATUS']
features = pd.DataFrame(df.drop(labels = ['CASE_STATUS'], axis = 1))

In [18]:
from sklearn import decomposition
from sklearn import preprocessing as pp
from sklearn import neighbors as knn
from sklearn import pipeline
from sklearn import model_selection
from sklearn import metrics
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTENC

# Build a preprocessor to scale numeric features and one hot encode categorical features.
numeric_features = ['NBR_WORKERS_REQUESTED', 
                    'BASIC_NUMBER_OF_HOURS', 
                    'BASIC_RATE_OF_PAY', 
                    'SUPERVISE_HOW_MANY', 
                    'NUM_OF_MONTHS_TRAINING',
                    'EMP_EXP_NUM_MONTHS',
                    'WORK_DAY_LENGTH']
categorical_features = ["AGENT_POC_EMP_REP_BY_AGENT", 
                        "SOC_CODE", 
                        "NAICS_CODE", 
                        "FULL_TIME_POSITION", 
                        "NATURE_OF_TEMPORARY_NEED", 
                        "EDUCATION_LEVEL", 
                        "SWA_NAME", 
                        "CITY_MATCH", 
                        "STATE_MATCH", 
                        "DAYTIME_WORK", 
                        "HAS_OVERTIME"]

# Get indices of numeric and categorical feature columns in dataframe.
numeric_feature_names = set(numeric_features)
categorical_feature_names = set(categorical_features)

numeric_feature_indices = []
categorical_feature_indices = []

for index, column_name in enumerate(features.columns):
    if column_name in numeric_feature_names:
        numeric_feature_indices.append(index)
    elif column_name in categorical_feature_names:
        categorical_feature_indices.append(index)

# Scale numeric features.
numeric_transformer = pipeline.Pipeline(steps=[
    ('scaler', pp.StandardScaler())])

# One-hot encode categorical features.
categorical_transformer = pipeline.Pipeline(steps=[
    ('onehot', pp.OneHotEncoder(sparse = False, handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_feature_indices),
        ('cat', categorical_transformer, categorical_feature_indices)])
        
random_state_val = 123
smote = SMOTENC(sampling_strategy = 1, random_state = random_state_val, categorical_features = categorical_feature_indices)

Here, we'll be determining the best parameters for Decision Tree, K Nearest Neighbors, SVM, Random Forest Classifier, and MLP Classifier.

In [19]:
# Decision Tree
tuned_parameters = {
    'tree__max_depth': [5, 10, 15, 20],
    'tree__min_samples_leaf': [5, 10, 15, 20], 
    'tree__max_features': [5, 10, 15],
    'tree__criterion': ['gini', 'entropy']
}
pipe = Pipeline(steps = [('smote', smote), ('preprocess', preprocessor), ('dim', decomposition.PCA()), ('tree', tree.DecisionTreeClassifier())])
tree_model = GridSearchCV(pipe, param_grid=tuned_parameters, scoring='accuracy', cv=5)
tree_model.fit(features, labels)
print(tree_model.best_params_)
print(tree_model.best_score_)

{'tree__max_depth': 10}
0.8140672782874617


In [11]:
# K Nearest Neighbors
tuned_parameters = {
    'knn__n_neighbors': list(range(1, 25))
}
pipe = Pipeline(steps = [('smote', smote), ('preprocess', preprocessor), ('dim', decomposition.PCA()), ('knn', neighbors.KNeighborsClassifier())])
knn_model = GridSearchCV(pipe, param_grid=tuned_parameters, scoring='accuracy', cv=5)
knn_model.fit(features, labels)
print(knn_model.best_params_)
print(knn_model.best_score_)

{'knn__n_neighbors': 1}
0.8844548774126239


In [12]:
# Support Vector Machine
tuned_parameters = {
    'svm__C': [0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2],
    'svm__kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}
pipe = Pipeline(steps = [('smote', smote), ('preprocess', preprocessor), ('dim', decomposition.PCA()), ('svm', svm.SVC(gamma='auto'))])
svm_model = GridSearchCV(pipe, param_grid=tuned_parameters, scoring='accuracy', cv=5)
svm_model.fit(features, labels)
print(svm_model.best_params_)
print(svm_model.best_score_)

{'svm__C': 2, 'svm__kernel': 'rbf'}
0.8172491740566858


In [13]:
# Neural Network
tuned_parameters = {
    'net__activation': ['logistic', 'tanh', 'relu'],
    'net__solver': ['sgd', 'adam']
}
pipe = Pipeline(steps = [('smote', smote), ('preprocess', preprocessor), ('dim', decomposition.PCA()), ('net', neural_network.MLPClassifier())])
net_model = GridSearchCV(pipe, param_grid=tuned_parameters, scoring='accuracy', cv=5)
net_model.fit(features, labels)
print(net_model.best_params_)
print(net_model.best_score_)

warnings.simplefilter("default")

{'net__activation': 'relu', 'net__solver': 'adam'}
0.8573291601460615


In [14]:
# Random Forest Classifier
tuned_parameters = {
    'rfc__max_depth': list(range(35, 56)),
    'rfc__min_samples_leaf': list(range(8, 13, 2)),
    'rfc__max_features': ['sqrt', 'log2']
}

pipe = Pipeline(steps = [('smote', smote), ('preprocess', preprocessor), ('dim', decomposition.PCA()), ('rfc', ensemble.RandomForestClassifier(n_estimators=10))])
rfc_model = GridSearchCV(pipe, param_grid=tuned_parameters, scoring='accuracy', cv=5)
rfc_model.fit(features, labels)
print(rfc_model.best_params_)
print(rfc_model.best_score_)

{'rfc__max_depth': 37, 'rfc__max_features': 'sqrt', 'rfc__min_samples_leaf': 8}
0.8755868544600939


Now, we create our classifiers based on the best tuned parameters.

We use these classifiers to create an ensemble voting classifier, hosting a Decision Tree, Random Forest, KNN, Gaussian Naive Bayes, Support Vvector Machine, and Neural Network classifier.

In [15]:
classifiers = []

# Classifiers.
tree = tree.DecisionTreeClassifier(criterion='entropy', max_depth=20, max_features=15, min_samples_leaf=5)
classifiers.append(('tree', tree))

rfc = ensemble.RandomForestClassifier(n_estimators=10, max_depth=37, max_features='sqrt', min_samples_leaf=8)
classifiers.append(('rfc', rfc))

knn = neighbors.KNeighborsClassifier(n_neighbors=1)
classifiers.append(('knn', knn))

nb = naive_bayes.GaussianNB()
classifiers.append(('nb', nb))

sv = svm.SVC(gamma='auto', C=2.0, kernel='rbf')
classifiers.append(('svm', sv))

net = neural_network.MLPClassifier(activation='relu', solver='adam')
classifiers.append(('neural_network', net))

In [16]:
# Results from the Individual Classifiers.

warnings.simplefilter("ignore")

for name, cl in classifiers:
    pipe = Pipeline(steps = [('smote', smote), ('preprocess', preprocessor), ('dim', decomposition.PCA()), ('classifier', cl)])
    pr = model_selection.cross_val_predict(pipe, features, labels, cv=5)
    
    print(cl)
    print(metrics.confusion_matrix(labels, pr))
    print(metrics.classification_report(labels, pr))
    
warnings.simplefilter("default")

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=20,
                       max_features=15, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
[[4933  818]
 [ 994 4757]]
              precision    recall  f1-score   support

           0       0.83      0.86      0.84      5751
           1       0.85      0.83      0.84      5751

    accuracy                           0.84     11502
   macro avg       0.84      0.84      0.84     11502
weighted avg       0.84      0.84      0.84     11502

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=37, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
            

In [17]:
voting = ensemble.VotingClassifier(classifiers)

pipe = Pipeline(steps = [('smote', smote), ('preprocess', preprocessor), ('dim', decomposition.PCA()), ('classifier', voting)])

warnings.simplefilter("ignore")
pr = model_selection.cross_val_predict(pipe, features, labels, cv=5)
warnings.simplefilter("default")
    
print(voting)
print(metrics.confusion_matrix(labels, pr))
print(metrics.classification_report(labels, pr))

VotingClassifier(estimators=[('tree',
                              DecisionTreeClassifier(class_weight=None,
                                                     criterion='entropy',
                                                     max_depth=20,
                                                     max_features=15,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=5,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort=False,
                                                     random_state=None,
                                                     splitter='best')),
     